# SWOT hydrology products time series examples

The Surface Water and Ocean Topography (SWOT) mission aims to provide valuable data and information about the world's oceans and its terrestrial surface water such as lakes, rivers, and wetlands. SWOT is being developed jointly by NASA and Centre National D'Etudes Spatiales (CNES), with contributions from the Canadian Space Agency (CSA) and United Kingdom Space Agency (UKSA). SWOT launched on the 16th of December of 2022 for its calibration-validation phase and entered its science 21-day repeat orbit in August 2023. More information can be found on [PO.DAAC SWOT website](https://podaac.jpl.nasa.gov/SWOT).

This notebook shows how to create and export time series of SWOT hydrology parameters for [river](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_RiverSP_2.0) and [lake](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_LakeSP_2.0) products. Additional information is available on the [SWOT Data User Handbook](https://www.earthdata.nasa.gov/s3fs-public/2024-06/D-109532_SWOT_UserHandbook_20240502.pdf).

## Libraries

In [ ]:
import json
import requests
import datetime
import numpy as np
import pandas as pd
import hvplot.pandas
import geopandas as gpd
from io import StringIO
from matplotlib import colormaps as cmaps
import matplotlib.colors as mcolors
from ipywidgets import Box, VBox, HBox, Label, HTML, Checkbox
from ipyleaflet import Map, GeoJSON, LayerGroup, DrawControl, WidgetControl, basemap_to_tiles

## HydroFeature Class and Subclasses
The `HydroFeature` class and its subclasses, `Lake` and `Reach`, are designed to manage and visualize hydrological data. By initializing these classes, we set up the infrastructure required for further analysis and visualization in the notebook. These classes allow us to interact with WFS and [Hydrocron API](https://podaac.github.io/hydrocron/overview.html), fetch relevant data, and perform operations such as querying time series data and displaying features on a map.

**Key Methods:**

- **`__init__(self, bbox)`**  
  Initializes with a bounding box (`bbox`) and sets up the WFS URL. Placeholder attributes such as `type_name`, `id_field`, and `feature` are initialized for subclass-specific values.

- **`get_features(self)`**  
  Fetches features from the [Prior River Database SWORD](https://catalogue.theia.data-terra.org/meta/SWOT_PRIOR_RIVER_DATABASE) or the [Prior Lake Database (PLD)](https://catalogue.theia.data-terra.org/meta/SWOT_PRIOR_LAKE_DATABASE) through a WFS request based on a bounding box.

- **`query_hydrocron(self, query_url, feature_id, start_time, end_time, fields)`**  
  Queries the Hydrocron API for time series data of the features identified in the area of interest, returning a list of data frames containing the queried data.

**Subclasses**

**`Lake` Class**  
Inherits from `HydroFeature`. Tailored for handling lake-specific features. Initializes with attributes specific to lakes, such as `type_name`, `id_field`, and `feature`.

**`Reach` Class**  
Inherits from `HydroFeature`. Designed for handling river reach features. Sets attributes specific to rivers, including `type_name`, `id_field`, and `feature`.

#### Using the [Hydrocron API](https://podaac.github.io/hydrocron/overview.html) to query time series data

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.13381966.svg)](https://doi.org/10.5281/zenodo.13381966)

Hydrocron is an API that repackages hydrology datasets from the Surface Water and Ocean Topography (SWOT) satellite into formats that make time-series analysis easier.

SWOT data is archived as individually timestamped shapefiles, which would otherwise require users to perform multiple file IO operations per river or lake feature to view the data as a timeseries. Hydrocron makes this possible with a single API call, facilitating creating analysis ready dataframes of the hydrological parameters from the [SWOT Level 2 River Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_RiverSP_2.0) and the [SWOT Level 2 Lake Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_LakeSP_2.0).


In [2]:
class HydroFeature:
    def __init__(self, bbox):
        self.wfs_url = "https://hydroweb.next.theia-land.fr/geoserver/REF_DATA/ows"
        self.bbox = bbox
        self.type_name = None
        self.id_field = None
        self.feature = None

    def get_features(self):
        """Fetch features from a WFS service using a bounding box.

        Returns
        -------
        tuple: (list, GeoDataFrame) - List of IDs and GeoDataFrame of the features
        """
        if self.bbox is None:
            raise ValueError("Bounding box (bbox) is not defined.")
        
        # Format bbox parameter for the WFS request
        bbox_str = f"{self.bbox[0]},{self.bbox[1]},{self.bbox[2]},{self.bbox[3]},EPSG:4326"
        
        # Parameters for GetFeature request
        params = {
            "service": "WFS",
            "version": "2.0.0",
            "request": "GetFeature",
            "typeName": self.type_name,  # Layer name set by subclass
            "outputFormat": "application/json",  # Requesting the response in JSON format
            "bbox": bbox_str  # Bounding box
        }
        
        try:
            # Send the request
            response = requests.get(self.wfs_url, params=params)
            
            # Check if the request was successful
            response.raise_for_status()  # Raise an HTTPError for bad responses
            
            # Parse the response as JSON
            feature_info = response.json()
            
            # Extract IDs from features based on the id_field
            ids = [feature['id'].split('.')[-1] for feature in feature_info['features']]
            
            # Convert JSON features to GeoDataFrame
            gdf = gpd.GeoDataFrame.from_features(feature_info['features'])
            
            return ids, gdf
        
        except requests.RequestException as e:
            print(f"Error: {e}")
            return [], gpd.GeoDataFrame()
        
    def visualize_features(self, gdf):
        """Visualize hydrological features on a map with checkboxes for toggling visibility."""
        if gdf.empty:
            print("No features to display.")
            return
        
        # Center the map around the bounding box center
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
        center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]  # (lat, lon)
        m = Map(center=center, zoom=9, scroll_wheel_zoom=True)

        # Layer group to hold all GeoJSON layers
        layer_group = LayerGroup()

        # Generate a color map
        cmap = cmaps.get_cmap('tab10').resampled(len(gdf))

        # Add each feature as a separate GeoJSON layer with color-coded checkboxes
        checkboxes = []
        for i, feature in enumerate(gdf.iterfeatures()):
            feature_id = feature['properties'].get(self.id_field)
            color = mcolors.to_hex(cmap(i))
            geojson_layer = GeoJSON(data=feature, style={'color': color, 'weight': 2})
            layer_group.add_layer(geojson_layer)

            # Create a colored square and checkbox with ID
            color_square = f"<div style='width: 15px; height: 15px; background-color: {color}; display: inline-block;'></div>"
            checkbox = Checkbox(value=True, indent=False)

            # Combine the colored square, checkbox, and feature ID into a single label
            label = f"<div style='display: inline-block; vertical-align: middle; margin-left: 5px; width: 100px'>{color_square} {feature_id}</div>"

            # Use HBox to place the checkbox and label next to each other
            checkbox_with_color = HBox([HTML(value=label), checkbox], layout={
                'display': 'flex',        # Flexbox layout to keep items in a row
                'align-items': 'center',  # Vertically center items within the HBox
                'width': '140px',         # Width to fit content
            })
                
            checkboxes.append((checkbox, geojson_layer, checkbox_with_color))

        # Add the layer group to the map
        m.add_layer(layer_group)

        # Function to toggle layers on checkbox change
        def toggle_layer(change, layer):
            layer.visible = change['new']

        # Attach the toggle functionality to each checkbox
        checkbox_widgets = []
        for checkbox, layer, checkbox_with_color in checkboxes:
            checkbox.observe(lambda change, layer=layer: toggle_layer(change, layer), 'value')
            checkbox_widgets.append(checkbox_with_color)

        # Create a VBox for checkboxes to toggle layers and add it to the top-right corner
        checkbox_vbox = VBox(checkbox_widgets, layout={
            'overflow_y': 'scroll',   # Enable vertical scrolling
            'display': 'flex',        # Flexbox layout to keep items in a column
            'align-items': 'center',  # Vertically center items within the VBox
            'padding': '5px',         # Add some padding inside the box
            'border': '1px solid black',  # Optional: Add a border around the checkbox container
            'width': '160px',         # Fixed width for the checkbox container
        })
        checkbox_control = WidgetControl(widget=checkbox_vbox, position='topright')
        m.add_control(checkbox_control)

        # Display the map
        display(m)

    def query_hydrocron(self, query_url, feature_id, start_time, end_time, fields):
        """Query Hydrocron for reach/lake-level time series data.
        
        Parameters
        ----------
        query_url : str
            URL to use to query FTS.
        feature_id : str
            Identifier for the feature (reach_id or lake_id).
        start_time : str
            Time to start query (ISO format).
        end_time : str
            Time to end query (ISO format).
        fields : str
            Comma-separated list of fields to return in query response.
        
        Returns
        -------
        pd.DataFrame
            DataFrame containing query results.
        """
        
        params = {
            "feature": self.feature,  # Dynamically set based on subclass
            "feature_id": feature_id,
            "output": "csv",
            "start_time": start_time,
            "end_time": end_time,
            "fields": fields
        }
        results = requests.get(query_url, params=params)

        # Check if the request was successful
        if results.status_code == 200:
            print(f"Success: Retrieved data for {self.feature} ID {feature_id}")
            if "results" in results.json().keys():
                results_csv = results.json()["results"]["csv"]
                df = pd.read_csv(StringIO(results_csv))
            else:
                df = self.create_empty_dataframe(feature_id)
        else:
            print(f"Error: Unable to retrieve data for {self.feature} ID {feature_id}, status code {results.status_code}")
            print(results.json())
            df = self.create_empty_dataframe(feature_id)
        
        return df

    def create_empty_dataframe(self, feature_id):
        """Create an empty DataFrame for cases where no data is returned for a feature identifier."""
        empty_df = pd.DataFrame({
            self.id_field: np.int64(feature_id),
            "time_str": datetime.datetime(1900, 1, 1).strftime("%Y-%m-%dT%H:%M:%S"),
            "wse": -999999999999.0,
            "wse_units": "m"
        }, index=[0])
        return empty_df
    
    def plot_results(self, results):
        """
        Plot the results of a set of time series data for multiple reach/lake identifiers.

        Parameters:
        - results (list of pandas.DataFrame): A list of DataFrames containing time series data for each identifier.

        Returns:
        - combined_plot (hvplot object): A combined plot of line and scatter plots for each identifier,
                                         with the x-axis representing time and the y-axis representing the water surface elevation.
        """
        # Load DataFrame results into a single Pandas DataFrame
        df = pd.concat(results, ignore_index=True)

        # Remove fill values for missing observations
        df = df.loc[(df["wse"] != -999999999999.0)]

        # Convert time_str to datetime format
        df.time_str = pd.to_datetime(df.time_str)

        # Plot results
        line_plot = df.hvplot(x="time_str", y="wse", by=self.id_field, kind="line", persist=True)
        line_plot.opts(xrotation=90, xlabel="Date", ylabel="Water Surface Elevation (m)")

        scatter_plot = df.hvplot(x="time_str", y="wse", by=self.id_field, kind="scatter", persist=True)
        combined_plot = line_plot * scatter_plot

        return combined_plot

class Lake(HydroFeature):
    def __init__(self, bbox):
        super().__init__(bbox)
        self.type_name = "swot_prior_lake_db"  # Lake specific type name
        self.id_field = "lake_id"  # Field to extract IDs for lakes
        self.feature = "PriorLake"

class Reach(HydroFeature):
    def __init__(self, bbox):
        super().__init__(bbox)
        self.type_name = "swot_prior_river_db"  # River specific type name
        self.id_field = "reach_id"  # Field to extract IDs for rivers
        self.feature = "Reach"

### Define your bounding box
Define your bounding box by drawing a rectangle to select the area of interest. `bbox` is a global variable and it is automatically updated to reflect the coordinates of the selected area. This bounding box will be used to fetch and analyze features within the specified geographic region.

In [ ]:
m = Map(center=(52.1326, 5.2913), zoom=4, scroll_wheel_zoom=True)

# Add a drawing control to the map
draw_control = DrawControl()

# Set drawing mode to rectangle
draw_control.rectangle = {
    "shapeOptions": {
        "color": "#ff0000",
        "weight": 4
    }
}

# Add the draw control to the map
m.add_control(draw_control)

# Widget to display bounding box coordinates
coords_label = Label()

bbox=None
# Callback function to capture and display bounding box coordinates
def handle_draw(self, action, geo_json):
    global bbox
    coords = geo_json['geometry']['coordinates'][0]
    min_lon, min_lat = coords[0]  # Southwest corner
    max_lon, max_lat = coords[2]  # Northeast corner
    bbox = (min_lon, min_lat, max_lon, max_lat)
    coords_label.value = f"bbox = {bbox}"

# Bind the callback function to the draw event
draw_control.on_draw(handle_draw)

# Display the map and coordinates
display(VBox([m, coords_label]))


### River Example
Using a WFS on the [SWOT Prior River Database SWORD](https://catalogue.theia.data-terra.org/meta/SWOT_PRIOR_RIVER_DATABASE) available through the Theia Data and Services centre for continental surfaces catalogue, extracting the reach IDs within your area of interest. 

The SWOT Prior River Database SWORD dataset provides high-resolution river nodes (200 m) and reaches (~10 km) with attached hydrologic variables (water surface elevation, width, slope, etc.) as well as a consistent topological system for global rivers 100 m wide and greater. The dataset combines multiple global river- and satellite-related datasets to define the nodes and reaches used in the [SWOT Level 2 River Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_RiverSP_2.0). See the [Product Description Document](http://gaia.geosci.unc.edu/SWORD/SWORD_ProductDescription_v16.pdf) and [Altenau et al. (2021)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2021WR030054) for additional information. The latest release of SWORD may be explored and downloaded on the [dedicated dashboard](https://www.swordexplorer.com/).

In [ ]:
# Initialize Reach subclass to get river features
river = Reach(bbox)
reach_ids, river_gdf = river.get_features() 

# Print and visualize the reach IDs as characterized in the SWORD database
print("List of river reach IDs:")
print(reach_ids)

river.visualize_features(river_gdf)

The hydrological parameters from the [SWOT Level 2 River Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_RiverSP_2.0) as described in the documentation can be modified in *fields*, with data field requests described in the [Hydrocron API endpoints](https://podaac.github.io/hydrocron/timeseries.html#request-parameters).

In [ ]:
# Parameters to start the query to the Hydrocron API
HYDROCRON_URL = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries"
start_time = "2023-02-01T00:00:00Z" 
end_time = "2024-09-01T00:00:00Z"
fields = "reach_id,time_str,wse,wse_u,reach_q,slope,width,river_name,geometry,dschg_c" # Adjust for your parameters of interest

# Query for river reach data
river_results = []
for reach in reach_ids:
    river_df = river.query_hydrocron(HYDROCRON_URL, reach, start_time, end_time, fields)
    river_results.append(river_df)

# Visualize Water Surface Elevation (WSE)
river.plot_results(river_results)

#### Exporting as a CSV

In [ ]:
for index, df in enumerate(river_results):
    # Extract the reach_id to use as part of the filename
    reach_id = df['reach_id'].iloc[0]
    
    # Define a file name using the reach_id
    file_name = f"timeSeries_reach{reach_id}.csv"
    
    # Export DataFrame to CSV
    df.to_csv(file_name, index=False)
    print(f"DataFrame for reach_id {reach_id} exported to {file_name}")

### Lake Example
Using a WFS on the [SWOT Prior Lake Database](https://catalogue.theia.data-terra.org/meta/SWOT_PRIOR_LAKE_DATABASE) available through the Theia Data and Services centre for continental surfaces catalogue, extracting the lake IDs within your area of interest. 

The SWOT Prior Lake Database dataset provides a global inventory of lakes and reservoirs with polygons and other useful metadata. The database provides prior data on known lakes, making it possible to link SWOT lake observations over time and to compute storage change.

It is generated by combining multiple global and regional hydrographic databases into one congruent product. The current version is mainly based on the CIRCA-2015 (Copyright UCLA) lake water extent map but relies also on several other datasets, including the *SWOT Prior River Database SWORD* which is therefore compatible. It is used to generate key hydrology parameters in the [SWOT Level 2 Lake Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_LakeSP_2.0). For further information visit the [Auxiliary Data Description Document](https://hydroweb.next.theia-land.fr/docs/collections/resources/swot/SWOT-IS-CDM-1944-CNES_AuxData_LakeDatabase_20231208_RevB_signed.pdf).

In [ ]:
# Initialize Lake subclass to get lake features
lake = Lake(bbox)
lake_ids, lake_gdf = lake.get_features()

# Print and visualize the lake IDs as characterized in the Prior Lake Database
print("List of lake IDs:")
print(lake_ids)

lake.visualize_features(lake_gdf)

The hydrological parameters from the [SWOT Level 2 Lake Single-Pass Vector Data Product](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_HR_LakeSP_2.0) as described in the documentation can be modified in *fields*, with data field requests described in the [Hydrocron API endpoints](https://podaac.github.io/hydrocron/timeseries.html#request-parameters).

In [ ]:
# Create queries that return Pandas.DataFrame objects
HYDROCRON_URL = "https://soto.podaac.earthdatacloud.nasa.gov/hydrocron/v1/timeseries"
start_time = "2023-02-01T00:00:00Z"
end_time = "2024-08-24T00:00:00Z"
fields = "lake_id,time_str,wse,wse_u,area_total,quality_f,lake_name"
lake_results = []
for i in lake_ids:
    lake_df = lake.query_hydrocron(HYDROCRON_URL, i, start_time, end_time, fields)
    lake_results.append(lake_df)
    
lake.plot_results(lake_results)

#### Exporting as a CSV

In [ ]:
for index, df in enumerate(lake_results):
    # Extract the lake_id to use as part of the filename
    lake_id = df['lake_id'].iloc[0] 
    
    # Define a file name using the reach_id
    file_name = f"timeSeries_lake{lake_id}.csv"
    
    # Export DataFrame to CSV
    df.to_csv(file_name, index=False)
    print(f"DataFrame for lake_id {lake_id} exported to {file_name}")

Further tutorials on SWOT data and its capabilities can be found in the [SWOT Data Tutorials by PO.DAAC](https://podaac.github.io/tutorials/quarto_text/SWOT.html) and in the [SWOT Community Repository](https://swot-community.github.io/SWOT-galleries/).

